<center> <img src = https://upload.wikimedia.org/wikipedia/commons/thumb/7/79/HeadHunter_logo.png/640px-HeadHunter_logo.png width=300px >

# <center> Проект: Анализ вакансий из HeadHunter

In [2]:
import pandas as pd 
import psycopg2

In [348]:
file = open('connection_parameters.txt') #Открытие параметров для соедиения
list_connection = [line.rstrip('\n') for line in file]
DBNAME = list_connection[0]
USER = list_connection[1]
PASSWORD = list_connection[2]
HOST = list_connection[3]
PORT = int(list_connection[4])

In [353]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)


## <center> 1. Предварительный анализ данных

**1) Запрос, который посчитает количество вакансий в нашей базе**

In [374]:
# текст запроса
query_1_1 = f'''select count(*) cnt_vac  
                from VACANCIES
'''
        

In [375]:
# результат запроса
cnt_vac = pd.read_sql_query(query_1_1, connection)
cnt_vac['cnt_vac'][0] #количество вакансий в базе

C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\3073949867.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cnt_vac = pd.read_sql_query(query_1_1, connection)


49197

**2) Запрос, который посчитает количество работадателей в нашей базе**

In [151]:
query_1_2 = f'''select count(*) cnt_employers
                from EMPLOYERS
'''

In [152]:
cnt_employers = pd.read_sql_query(query_1_2, connection)
cnt_employers['cnt_employers'][0] #количество работадателей в базе


C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\1923868411.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cnt_employers = pd.read_sql_query(query_1_2, connection)


23501

**3) Запрос, который посчитает количество регионов в нашей базе**

In [149]:
query_1_3 = f'''select count(distinct name) cnt_regions
                from AREAS
'''

In [150]:
cnt_regions = pd.read_sql_query(query_1_3, connection)
cnt_regions['cnt_regions'][0] #Количество регионов в базе


C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\151406758.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cnt_regions = pd.read_sql_query(query_1_3, connection)


1362

**4) Запрос, который посчитает количество сфер деятельности в нашей базе**

In [153]:
query_1_4 = f'''select count(distinct name) areas_of_activity
                from INDUSTRIES
'''

In [159]:
areas_of_activity = pd.read_sql_query(query_1_4, connection)
areas_of_activity['areas_of_activity'][0] #Количество регионов в базе


C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\2494791268.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  areas_of_activity = pd.read_sql_query(query_1_4, connection)


294

##### Выводы по предварительному анализу данных:

* В базе 49197 вакансий

* В базе 23501 работадателей

* Данных достаточно для анализа

* В базе регионы не только из РФ

## <center> 2. Детальный анализ вакансий

**1) Запрос, который позволит узнать топ-5 регионов по количеству вакансий**

In [184]:
n = 5
query_2_1 = f'''select a.name, count(v.id) cnt_area
           from VACANCIES v
           join AREAS a on area_id = a.id
           group by a.id
           order by 2 desc
           limit {n}'''

In [185]:
cnt_area = pd.read_sql_query(query_2_1, connection)
list(cnt_area['name'])

C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\1215467203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cnt_area = pd.read_sql_query(query_2_1, connection)


['Москва', 'Санкт-Петербург', 'Минск', 'Новосибирск', 'Алматы']

**2) Запрос, который позволит узнать, у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой?**

In [165]:
query_2_2 = f'''select count(*) filled_vac
                from VACANCIES 
                where salary_from is not null or salary_to is not null
'''

In [166]:
filled_vac = pd.read_sql_query(query_2_2, connection)
filled_vac['filled_vac'][0]

C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\742889117.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  filled_vac = pd.read_sql_query(query_2_2, connection)


24073

**3) Запрос, который позволит узнать, средние значения для нижней и верхней границы зарплатной вилки**

In [167]:
query_2_3 = '''select ROUND(avg(salary_from)) avg_salary_from, ROUND(avg(salary_to)) avg_salary_to
               from VACANCIES
'''

In [178]:
salary_fork = pd.read_sql_query(query_2_3, connection)
print(salary_fork['avg_salary_from'][0])
print(salary_fork['avg_salary_to'][0])

71065.0
110537.0


C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\2151410556.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  salary_fork = pd.read_sql_query(query_2_3, connection)


**4) Запрос, который позволит узнать, количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. В качестве ответа приводится вторая пара по популярности**

In [376]:
query_2_4 = '''select schedule, employment, count(id)
           from VACANCIES
           group by 1, 2
           order by 3 desc
           offset 1 limit 1
'''

In [377]:
cnt_empl_sch = pd.read_sql_query(query_2_4, connection)
cnt_empl_sch

C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\1947912757.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cnt_empl_sch = pd.read_sql_query(query_2_4, connection)


,schedule,employment,count
0,Удаленная работа,Полная занятость,7802


**5) Запрос, выводящий требуемый опыт работы от количества вакансий в порядке возрастания**

In [186]:
query_2_5 = '''select experience, count(id) cnt_vacancies
           from VACANCIES
           group by experience
           order by 2
'''

In [189]:
experience = pd.read_sql_query(query_2_5, connection)
list(experience['experience'])

C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\2173679686.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  experience = pd.read_sql_query(query_2_5, connection)


['Более 6 лет', 'Нет опыта', 'От 3 до 6 лет', 'От 1 года до 3 лет']

**6) Заброс, показывающий зависимость средней зарплаты от рабочего графика**

In [ ]:
query_5_1 = '''select schedule, round(avg((coalesce(null, salary_from, salary_to)  + coalesce(null, salary_to, salary_from)) / 2)) avg_sal_schedule
               from vacancies
               where salary_from is not null or salary_to is not null
               group by schedule
'''

In [364]:
avg_sal_schedule = pd.read_sql_query(query_5_1, connection)
avg_sal_schedule

C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\1446242491.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  avg_sal_schedule = pd.read_sql_query(query_5_1, connection)


,schedule,avg_sal_schedule
0,Сменный график,44532.0
1,Вахтовый метод,123790.0
2,Гибкий график,100693.0
3,Удаленная работа,101323.0
4,Полный день,80126.0


##### Выводы по анализу вакансий:

* Самое большое количество вакансий находится в городе Москва

* Около половины вакансий не имеют верхней или нижней границы оплаты труда в рублях. Возможно это связано с тем, что компания боится спугнуть потенциального работодателя

* Среднее значение нижней границы зарплатной вилки - 71065 рублей, верхней - 110537

* Самые популярные пары "тип рабочего графика - тип трудоустройства": Полный день — Полная занятость и Удалённая работа — Полная занятость

* Самый требуемый опыт работы - от 1 года до 3 лет, самый редкий - более 6 лет. Так как людей, которые дошли до большого стажа в конкретой работе гораздо меньше, чем начинающих. Также, чем меньше требуемый опыт, тем проще найти сотрудника

* Максимальная средняя зарплата при вахтовом графике работы


## <center> 3. Анализ работодателей

**1) Запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.**

In [190]:
query_3_1 = '''(select e.name, count(e.id) 
                from EMPLOYERS e
                join VACANCIES v on e.id = v.employer_id
                group by e.name
                order by count(e.id) desc
                limit 1)

                union all

                (select e.name, count(e.id)
                from EMPLOYERS e
                join VACANCIES v on e.id = v.employer_id
                group by e.name
                order by count(e.id) desc
                offset 4 limit 1)
'''

In [195]:
employers = pd.read_sql_query(query_3_1, connection)
print(employers['name'][0])
print(employers['name'][1])

Яндекс
Газпром нефть


C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\1119753015.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employers = pd.read_sql_query(query_3_1, connection)


**2) Запрос, который для каждого региона выведет количество работодателей и вакансий в нём (среди регионов, в которых нет вакансий)**

In [208]:
query_3_2 =  '''select a.name, count(distinct e.id) cnt_emp, count(distinct v.id) cnt_amp
                from areas a
                left join vacancies v on v.area_id = a.id              
                left join employers e on e.area = a.id
                where v.id is null
                group by a.id
                order by 2 desc
                limit 1
'''

In [209]:
cnt_emp_amp = pd.read_sql_query(query_3_2, connection)
cnt_emp_amp['name'][0]

C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\1115088539.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cnt_emp_amp = pd.read_sql_query(query_3_2, connection)


'Россия'

**3) Запрос, показывающий максимальное количество регионов, в которых работодатель публикует свои вакансии**

In [218]:
query_3_3 =  '''SELECT e.name, count(DISTINCT v.area_id) AS cnt_a
                FROM employers e
                JOIN vacancies v ON e.id=v.employer_id
                GROUP BY e.id
                ORDER BY cnt_a DESC
                LIMIT 1
'''

In [219]:
cnt_region = pd.read_sql(query_3_3, connection)
cnt_region['cnt_a'][0]

C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\2644597097.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cnt_region = pd.read_sql(query_3_3, connection)


181

**4) Запрос для подсчёта количества работодателей, у которых не указана сфера деятельности**

In [225]:
query_3_4 = '''select count(id) cnt_emp
               from EMPLOYERS e
               left join EMPLOYERS_INDUSTRIES ei on employer_id = e.id
               where employer_id is null'''

In [229]:
cnt_emp = pd.read_sql_query(query_3_4, connection)
cnt_emp['cnt_emp'][0]

C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\3896276666.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cnt_emp = pd.read_sql_query(query_3_4, connection)


8419

**5) Запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности**

In [332]:
query_3_5 = '''select e.name, count(industry_id)
               from EMPLOYERS e
               join EMPLOYERS_INDUSTRIES ei on employer_id = e.id
               group by id 
               having count(industry_id) =4
               order by e.name
               offset 2 limit 1
'''

In [333]:
query_name = pd.read_sql_query(query_3_5, connection)
query_name['name'][0]

C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\3721174881.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  query_name = pd.read_sql_query(query_3_5, connection)


'2ГИС'

**6) Запрос, показывающий у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения»**

In [242]:
query_3_6 = '''select i.name, count(employer_id) cnt_soft_develop
               from INDUSTRIES i 
               join EMPLOYERS_INDUSTRIES ei on industry_id = i.id
               where i.name = 'Разработка программного обеспечения'
               group by i.id
'''


In [243]:
cnt_soft_develop = pd.read_sql_query(query_3_6, connection)
cnt_soft_develop['cnt_soft_develop'][0]

C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\2310791923.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cnt_soft_develop = pd.read_sql_query(query_3_6, connection)


3553

**7) Список регионов-миллионников , в которых представлены вакансии компании "Яндекс", вместе с количеством вакансий в этих регионах**

In [365]:
tables = pd.read_html('https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8')
df = tables[1]
cities = tuple(df['Город']) # Города миллионеры
query_3_7 =  f'''select a.name, count(v.id)
                from vacancies v
                left join employers e on v.employer_id = e.id
                left join areas a on a.id = v.area_id
                where e.name = 'Яндекс' and a.name in {cities}
                group by a.name
                
                union all
                
                select 'Total', count((v.id))
                from vacancies v
                left join employers e on v.employer_id = e.id
                left join areas a on a.id = v.area_id
                where e.name = 'Яндекс' and a.name in {cities}
                order by 2 
'''

In [368]:
cnt_yan_city_m = pd.read_sql_query(query_3_7, connection)
cnt_yan_city_m

C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\265762831.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cnt_yan_city_m = pd.read_sql_query(query_3_7, connection)


,name,count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


**8) Количество вакансий в городах-миллионерах**

In [371]:
query_3_8 = f'''select a.name, count(v.id)
                from vacancies v
                left join employers e on v.employer_id = e.id
                left join areas a on a.id = v.area_id
                where  a.name in {cities}
                group by a.name
                
                union all
                
                select 'Total', count((v.id))
                from vacancies v
                left join employers e on v.employer_id = e.id
                left join areas a on a.id = v.area_id
                where  a.name in {cities}
                order by 2 
'''

In [373]:
cnt_vac_city_m = pd.read_sql_query(query_3_8, connection)
cnt_vac_city_m

C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\332801555.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cnt_vac_city_m = pd.read_sql_query(query_3_8, connection)


,name,count
0,Волгоград,456
1,Омск,617
2,Уфа,767
3,Пермь,771
4,Челябинск,786
5,Красноярск,847
6,Воронеж,1063
7,Ростов-на-Дону,1131
8,Самара,1144
9,Краснодар,1301


##### Выводы по анализу работадатей:

* Самый популярный работадатель - "Яндекс"

* "Яндекс" разместил свои вакансии в 181 регионе

* Треть работадотелей не указывают сферу деятельности

* У компании "2ГИС" указано 4 сферы деятельности

* У 3553 работодателей в качестве сферы деятельности указана «Разработка программного обеспечения»

* Среди городов-миллионеров наибольшее количество вакансий компании "Яндекс" в городе Москва

* В городах-миллионерах 485 вакансий компании "Яндекс"

* В городах-миллионерах находится около половины вакансий

# <center> 4. Предметный анализ

**1) Количество вакинсий, имеющих отношения к данным?**

In [255]:
query_4_1 ='''select count(*) vac_data
          from VACANCIES v
          where lower(name) like '%данн%' or lower(name) like '%data%'
'''

In [256]:
vac_data = pd.read_sql_query(query_4_1, connection)
print(vac_data['vac_data'][0])

1771


C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\3684118262.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vac_data = pd.read_sql_query(query_4_1, connection)


**2) Количество подходящих вакансий для начинающего дата-сайентиста**

In [257]:
query_4_2 ='''select count(*) cnt_jun_ds
            from VACANCIES v
            where (lower(name) like '%data scientist%' or lower(name) like '%data science%' or lower(name) like '%исследователь данных%' or 
            (name like '%ML%' and name not like '%HTML%') or lower(name) like '%machine learning%' or lower(name) like '%машинн%обучен%%')
            and (lower(name) like '%junior%' or experience like '%Нет опыта%' or employment like 'Стажировка')'''

In [275]:
cnt_jun_ds = pd.read_sql_query(query_4_2, connection)
cnt_jun_ds['cnt_jun_ds'][0]

C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\3402010046.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cnt_jun_ds = pd.read_sql_query(query_4_2, connection)


51

**3) Количество вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres**

In [100]:
query_4_3 =  '''select count(*) cnt_sql_ds
                from VACANCIES v
                where (lower(name) like '%data scientist%' or lower(name) like '%data science%' or lower(name) like '%исследователь данных%' or 
                (name like '%ML%' and name not like '%HTML%') or lower(name) like '%machine learning%' or lower(name) like '%машинн%обучен%%') 
                and (lower(key_skills) like '%sql%' or lower(key_skills) like '%postgres%')
'''

In [103]:
cnt_sql_ds = pd.read_sql_query(query_4_3, connection)
cnt_sql_ds['cnt_sql_ds'][0]

C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\928852559.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cnt_sql_ds = pd.read_sql_query(query_6_3, connection)


201

**4) Насколько популярен Python в требованиях работодателей к DS. Количество вакансий, в которых в качестве ключевого навыка указан Python.**

In [107]:
query_4_4 =  '''select count(*) cnt_py_ds
                from VACANCIES v
                where (lower(name) like '%data scientist%' or lower(name) like '%data science%' or lower(name) like '%исследователь данных%' or 
                (name like '%ML%' and name not like '%HTML%') or lower(name) like '%machine learning%' or lower(name) like '%машинн%обучен%%') 
                and (lower(key_skills) like '%python%')'''

In [109]:
cnt_py_ds = pd.read_sql_query(query_4_4, connection)
cnt_py_ds['cnt_py_ds'][0]

C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\25366536.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cnt_py_ds = pd.read_sql_query(query_6_4, connection)


351

**5) Количество ключевых навыков в среднем указывают в вакансиях для DS**

In [266]:
query_4_5 =  '''select  round(AVG(length(key_skills) - length(replace(key_skills, chr(9), '')) + 1), 2) cnt_avg_key_skill
                from VACANCIES v
                where (lower(name) like '%data scientist%' or lower(name) like '%data science%' or lower(name) like '%исследователь данных%' or 
                (name like '%ML%' and name not like '%HTML%') or lower(name) like '%machine learning%' or lower(name) like '%машинн%обучен%%')
'''

In [267]:
cnt_avg_key_skill = pd.read_sql_query(query_4_5, connection)
cnt_avg_key_skill['cnt_avg_key_skill'][0]

C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\1432531198.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cnt_avg_key_skill = pd.read_sql_query(query_4_5, connection)


6.41

**6) Запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта**

In [359]:
query_4_6 =  '''select  experience, round(avg((coalesce(null, salary_from, salary_to)  + coalesce(null, salary_to, salary_from)) / 2)) avg_salary
                from VACANCIES v
                where (lower(name) like '%data scientist%' or lower(name) like '%data science%' or lower(name) like '%исследователь данных%' or 
                (name like '%ML%' and name not like '%HTML%') or lower(name) like '%machine learning%' or lower(name) like '%машинн%обучен%%') 
                and (salary_from is not NUll or salary_to is not NUll) 
                group by experience
             '''

In [360]:
avg_salary = pd.read_sql_query(query_4_6, connection)
avg_salary[avg_salary['experience'] == 'От 3 до 6 лет']['avg_salary'][2]

C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\1998556985.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  avg_salary = pd.read_sql_query(query_4_6, connection)


243115.0

**7) Заброс, показывающий зависимость средней зарплаты дата-сайентиста от рабочего графика**

In [362]:
query_4_7 =  '''select  schedule, round(avg((coalesce(null, salary_from, salary_to)  + coalesce(null, salary_to, salary_from)) / 2)) avg_salary
                from VACANCIES v
                where (lower(name) like '%data scientist%' or lower(name) like '%data science%' or lower(name) like '%исследователь данных%' or 
                (name like '%ML%' and name not like '%HTML%') or lower(name) like '%machine learning%' or lower(name) like '%машинн%обучен%%') 
                and (salary_from is not NUll or salary_to is not NUll) 
                group by schedule
'''

In [363]:
ds_avg_sal_schedle = pd.read_sql_query(query_4_7, connection)
ds_avg_sal_schedle

C:\Users\rodni\AppData\Local\Temp\ipykernel_30472\1803103931.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ds_avg_sal_schedle = pd.read_sql_query(query_4_7, connection)


,schedule,avg_salary
0,Гибкий график,140500.0
1,Полный день,175726.0
2,Удаленная работа,190043.0


##### Выводы по предметному анализу:

* 1771 вакансий имеет отношение к данным

* Для начинающего дата-сайентиста имеется 51 вакансия

* Количество вакансий для дата-сайентиста, где в качестве ключевого навыка указан SQL или postgres - 201

* Количество вакансий для дата-сайентиста, где в качестве ключевого навыка указан Python - 351

*  В среднем в вакансиях для DS указывают около 6 ключевых навыков

*  Дата-сайентист с опытом работы от 3 до 6 лет может расчитывать на зарплату 243115 р.

* Для дата-сайнтистов наибольшая средняя зарпата при удаленном графике работы

In [378]:
connection.close() #закрытие соединения после окончания работы

# <center> Общий вывод по результатам анализа

Целью проекта было проанализировать вакансии из HeadHunter. На основании проведенного исследования, можно сделать следующие выводы: 

* Наибольшее количество вакансий в городе Москва

* Самые популярные графики работы в вакансиях - полный день и удаленная работа

Преимущество полного рабочего для в том, что один сотрудник любом случае продуктивнее, чем два сотрудника на половину рабочего дня. Преимущество удалённой работы для работодателя в том, что он может отбирать сотрудников вне зависимости от места жительства. Это повышает шансы найти специалистов с нужными навыками.

* Половина от общего количества вакансий находится в городах-миллионерах

Обуславливается тем, что люди из обычных городов или деревень стремятся переехать в города-миллионеры, поэтому в таких городах выше конкуренция, следовательно и больше работадателей

* Самый популярный работадатель - "Яндекс"

* Самая популярная сфера деятельности - разработка программного обеспечения

Объясняется это тем, что владельцы бизнеса стремятся к максимальной автоматизации, эффективности алгоритмов.